# Start

In [1]:
import pandas as pd

from aidd.batch.data_manager import get_provide_data, get_rename_columns
from aidd.batch.preprocessing import Preprocessing
from aidd.sys.utils import AiddException

# Load Data

In [2]:
pdata = get_provide_data()

[940a33983d55][2024-02-28 09:42:15.079987] 제공받은 데이터 불러오기 시작
[940a33983d55][2024-02-28 09:42:43.034814]   읽은 데이터: CONS, 데이터 크기: (19052, 143), 처리시간: 0:00:27.954602
[940a33983d55][2024-02-28 09:43:07.804938]   읽은 데이터: POLE, 데이터 크기: (38533, 63), 처리시간: 0:00:24.769959
[940a33983d55][2024-02-28 09:43:39.673338]   읽은 데이터: LINE, 데이터 크기: (40019, 77), 처리시간: 0:00:31.868273
[940a33983d55][2024-02-28 09:43:53.035455]   읽은 데이터: SL, 데이터 크기: (22632, 57), 처리시간: 0:00:13.361988
[940a33983d55][2024-02-28 09:43:53.035553] 제공받은 데이터 불러오기 종료, 최종 처리시간: 0:01:37.955595


# Pre-Processing

In [3]:
try:
    pp = Preprocessing(pdata)
except AiddException as ae:
    print(ae)
except Exception as e:
    print(f'SYS Error: {e}')

[1d45ab584729][2024-02-28 09:43:53.049974] 배치 데이터 전처리 시작
[64293d987818][2024-02-28 09:43:53.050094]   공사비 데이터 전처리 시작
[64293d987818][2024-02-28 09:43:53.801346]     공사비 데이터 1차 전처리 결과(데이터 크기): (15335, 16)
[64293d987818][2024-02-28 09:43:53.893020]   공사비 데이터 전처리 종료, 최종 처리시간: 0:00:00.842914
[394e141aafd8][2024-02-28 09:43:53.893116]   공비비 데이터별 설비 갯 수 계산 전처리 시작
[394e141aafd8][2024-02-28 09:43:54.077541]     공사비 데이터별 설비 갯 수 계산 후 데이터 크기: (14729, 19)
[394e141aafd8][2024-02-28 09:43:54.189445]   공비비 데이터별 설비 갯 수 계산 전처리 종료, 최종 처리시간: 0:00:00.296313
[ad88cba43d43][2024-02-28 09:43:54.189768]   전주 데이터 전처리 시작
[ad88cba43d43][2024-02-28 09:43:54.426532]     전주 데이터만 전처리한 결과(데이터 크기): (26921, 20)
[1d45ab584729][2024-02-28 09:43:54.426630] 배치 데이터 전처리 종료, 최종 처리시간: 0:00:01.376665


# Test

### Check Unique Data Type

In [5]:
type(pp._ppoledf.CONS_ID.unique())

numpy.ndarray

In [6]:
summation_columns = [col for col in pp._ppoledf.columns if col.startswith('POLE_')]

In [7]:
for col in summation_columns:
    print(col)

POLE_SHAPE_G
POLE_SHAPE_O
POLE_SHAPE_V
POLE_TYPE_1
POLE_TYPE_B
POLE_TYPE_C
POLE_TYPE_E
POLE_TYPE_H
POLE_TYPE_M
POLE_SPEC_6.0
POLE_SPEC_8.0
POLE_SPEC_10.0
POLE_SPEC_11.0
POLE_SPEC_12.0
POLE_SPEC_14.0
POLE_SPEC_16.0


In [8]:
ldf = pp.pdata['LINE']

In [9]:
select_columns = [
    'CONS_ID', 'COMP_ID', 'FROM_COMP_ID',
    'WIRING_SCHEME', 'LINE_TYPE_CD', 'LINE_SPEC_CD', 'LINE_PHASE_CD',
    'SPAN', 'NEUTRAL_TYPE_CD', 'NEUTRAL_SPEC_CD', 'COORDINATE'
]

In [10]:
ldf = ldf[select_columns]

In [12]:
## 코드형 데이터 One-Hoe Encoding
ldf = pd.get_dummies(
    ldf, 
    columns=[
        'WIRING_SCHEME', 'LINE_TYPE_CD', 'LINE_SPEC_CD', 'LINE_PHASE_CD',
        'NEUTRAL_TYPE_CD', 'NEUTRAL_SPEC_CD'
    ],
    prefix=[
        'WIRING_SCHEME', 'LINE_TYPE', 'LINE_SPEC', 'LINE_PHASE',
        'NEUTRAL_TYPE', 'NEUTRAL_SPEC'
    ]
)

In [13]:
ldf.columns.tolist()

['CONS_ID',
 'COMP_ID',
 'FROM_COMP_ID',
 'SPAN',
 'COORDINATE',
 'WIRING_SCHEME_13',
 'WIRING_SCHEME_41',
 'WIRING_SCHEME_43',
 'LINE_TYPE_AO',
 'LINE_TYPE_C1',
 'LINE_TYPE_C2',
 'LINE_TYPE_C4',
 'LINE_TYPE_D2',
 'LINE_TYPE_D4',
 'LINE_TYPE_EW',
 'LINE_TYPE_OC',
 'LINE_TYPE_OW',
 'LINE_SPEC_3.2',
 'LINE_SPEC_22.0',
 'LINE_SPEC_25.0',
 'LINE_SPEC_32.0',
 'LINE_SPEC_35.0',
 'LINE_SPEC_38.0',
 'LINE_SPEC_58.0',
 'LINE_SPEC_60.0',
 'LINE_SPEC_70.0',
 'LINE_SPEC_100.0',
 'LINE_SPEC_120.0',
 'LINE_SPEC_150.0',
 'LINE_PHASE_1',
 'LINE_PHASE_2',
 'LINE_PHASE_3',
 'NEUTRAL_TYPE_AL',
 'NEUTRAL_TYPE_AO',
 'NEUTRAL_TYPE_C1',
 'NEUTRAL_TYPE_OC',
 'NEUTRAL_TYPE_OW',
 'NEUTRAL_TYPE_WO',
 'NEUTRAL_TYPE_ZZ',
 'NEUTRAL_SPEC_0.0',
 'NEUTRAL_SPEC_22.0',
 'NEUTRAL_SPEC_32.0',
 'NEUTRAL_SPEC_35.0',
 'NEUTRAL_SPEC_38.0',
 'NEUTRAL_SPEC_58.0',
 'NEUTRAL_SPEC_60.0',
 'NEUTRAL_SPEC_95.0',
 'NEUTRAL_SPEC_100.0',
 'NEUTRAL_SPEC_150.0']